In [1]:
from pathlib import Path
from math import inf
from time import perf_counter
import tracemalloc

In [2]:
INPUT_FILE = Path.cwd().parent / "inputs" / "astar_in.txt"

### Helper Functions

This section defines utility functions that support the main algorithms. These include functions for displaying the puzzle state, checking if the current state matches the goal state, and performing basic operations like finding the position of the blank tile or generating possible moves. These helper functions are essential for implementing the search algorithms and heuristics that follow.

In [3]:
def parse_astar_in(path: Path):
    """ Parses the input file for A* algorithm and returns the start and goal grids as tuples."""

    with open(path, 'r') as f:
        lines = f.readlines()
    
    start, goal = False, False
    start_grid = []
    goal_grid = []
    for a in lines:
        if a.strip() == 'start':
            start = True
            continue
        
        if a.strip() == 'goal':
            goal = True
            continue
        
        if start and not goal:
            start_grid.extend(0 if x == "*" else int(x) for x in a.strip().replace(" ", ""))

        if goal:
            if len(start_grid)== 9:
                goal_grid.extend(0 if x == "*" else int(x) for x in a.strip().replace(" ", ""))
            else:
                print("Error: Start grid should have 9 elements before parsing goal grid.")
                break
    
    if start == False or goal == False:
        print("Error: Input file must contain 'start' and 'goal' sections.")
        print("Please check the input file format.")
        return None, None
    
    return tuple(start_grid), tuple(goal_grid)

In [4]:
def print_puzzle(state: tuple, N: int = 3, blank: int = 0):
    """ Prints the puzzle state in a readable format, replacing the blank tile with a space."""
    for r in range(N):
        row = state[r*N:(r+1)*N]
        print(" ".join(" " if x == blank else str(x) for x in row))
    print()

def print_path(path, N: int = 3, blank: int = 0):
    """ Prints the sequence of puzzle states in the path, showing each step clearly. """
    for i, state in enumerate(path):
        print(f"Step {i}:")
        print_puzzle(state, N=N, blank=blank)


In [5]:
def neighbors(state: tuple, N: int):
    """ Generates all valid neighboring states by sliding the blank tile in the four possible directions. """
    blank_tile_index = state.index(0)
    blank_tile_index_r, blank_tile_index_c = divmod(blank_tile_index, N)

    successors = []

    for direction_r, direction_c in [(-1,0),(1,0),(0,-1),(0,1)]:
        new_r, new_c = blank_tile_index_r+direction_r, blank_tile_index_c+direction_c
        if 0 <= new_r < N and 0 <= new_c < N:
            nz = new_r*N + new_c
            lst = list(state)
            lst[blank_tile_index], lst[nz] = lst[nz], lst[blank_tile_index]
            successors.append(tuple(lst))

    return successors

In [6]:
def reconstruct(parent, goal: tuple):
    """ Reconstructs the path from the start state to the goal state using the parent mapping."""
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = parent[cur]
    return path[::-1]

In [ ]:
def astar_algorithm(start: tuple, goal: tuple, heuristic_fn: callable):
    """ Implements the A* search algorithm to find the optimal path from start to goal state using the provided heuristic function."""

    # Step 1: initialization
    OPEN = []               # list of states ready for expansion
    OPEN_SET = set()        # for O(1) membership checks of OPEN
    CLOSED = set()          # expanded states

    Ns = int(len(start) ** 0.5)
    Ng = int(len(goal) ** 0.5)
    if Ns * Ns != len(start) or Ng * Ng != len(goal) or Ns != Ng:
        raise ValueError("Start and goal states must be same size and form an N×N puzzle.")
    N = Ns

    g = {start: 0}
    parent = {start: None}

    OPEN.append(start)
    OPEN_SET.add(start)
    nodes_generated = 1

    goal_positions = {tile: divmod(i, N) for i, tile in enumerate(goal)}

    # h_cache maps state -> heuristic value
    h_cache = {}
    s_cache = {}
    def h(state):
        if state not in h_cache:
            h_cache[state], s_cache[state] = heuristic_fn(state, goal, goal_positions, N)
        return h_cache[state], s_cache[state]
    
    while True:
        # Step 2: Check if OPEN is empty -> failure
        if not OPEN:
            raise Exception("No solution found.")
        
        # Step 3: pick n from OPEN with smallest f(n)=g(n)+h(n)
        # Tie-break in favor of goal only when f ties.
        n = min(OPEN, key=lambda s: (g[s] + h(s), 0 if s == goal else 1))
        
        OPEN.remove(n)
        OPEN_SET.remove(n)
        CLOSED.add(n)

        h_val = h(n)
        f_val = g[n] + h_val
        print(f"Expanding: g={g[n]}  h={h_val}  f={f_val} P={h_val - h_cache.get(parent[n], 0) if parent[n] else 'N/A'} S={s_cache.get(n, 'N/A')}")

        # Step 4: if n is goal -> return solution path by tracing parents       
        if n == goal:
            print("Nodes generated:", nodes_generated)
            return reconstruct(parent, n), N
        
        # Step 5: expand n, generate successors
        succ = neighbors(n, N)
        if not succ:
            continue  # "go immediately to 2"

        for ni in succ:
            nodes_generated += 1
            tentative_g = g[n] + 1  # unit step cost for 8-puzzle

            # Step 6: if ni not on OPEN or CLOSED -> add to OPEN, point back to n
            if ni not in OPEN_SET and ni not in CLOSED:
                g[ni] = tentative_g
                parent[ni] = n
                OPEN.append(ni)
                OPEN_SET.add(ni)

            # Step 7: if ni already on OPEN or CLOSED, keep the shorter path (smaller g => smaller f)
            elif tentative_g < g.get(ni, inf):
                g[ni] = tentative_g
                parent[ni] = n

                # if it was on CLOSED and improved, move back to OPEN
                if ni in CLOSED:
                    CLOSED.remove(ni)
                    OPEN.append(ni)
                    OPEN_SET.add(ni)
                # if it's already in OPEN_SET, nothing else needed (its g/parent got improved)

        # Step 8: loop continues

### Misplaced Tile Heuristic Function

The Misplaced Tile heuristic is a simple admissible heuristic for the 8-puzzle problem. It calculates the number of tiles that are not in their correct goal positions, excluding the blank tile. This heuristic provides a lower bound on the number of moves required to reach the goal, as each misplaced tile must be moved at least once. While not as informative as more sophisticated heuristics, it's computationally efficient and guarantees optimality when used with A* search.

In [8]:
def h_misplaced(state: tuple, goal: tuple, goal_positions: dict = None, N: int = 3):
    count = 0
    for s, g in zip(state, goal):
        if s != 0 and s != g:
            count += 1
    return count

In [9]:
start, goal = parse_astar_in(INPUT_FILE)

tracemalloc.start()
t0 = perf_counter()
path, N = astar_algorithm(start, goal, h_misplaced)

elapsed = perf_counter() - t0

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print_path(path, N=N)
print(f"time={elapsed:.4f}s  current={current/1e6:.2f}MB  peak={peak/1e6:.2f}MB")

Expanding: g=0  h=7  f=7
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=1  h=7  f=8
Expanding: g=2  h=6  f=8
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=2  h=7  f=9
Expanding: g=3  h=6  f=9
Expanding: g=3  h=6  f=9
Expanding: g=3  h=6  f=9
Expanding: g=2  h=8  f=10
Expanding: g=2  h=8  f=10
Expanding: g=3  h=7  f=10
Expanding: g=3  h=7  f=10
Expanding: g=3  h=7  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=6  f=10
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=3  h=8  f=11
Expanding: g=3  h=8  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=4  h=7  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11
Expanding: g=5  h=6  f=11


### Manhattan Distance Heuristic Function

The Manhattan Distance heuristic measures the total distance each tile needs to move to reach its goal position, calculated as the sum of horizontal and vertical distances for each tile (ignoring obstacles, hence "Manhattan" distance). This heuristic is admissible because it never overestimates the true cost to the goal, as each tile must move at least that many steps. It's more accurate than the Misplaced Tile heuristic and often leads to faster search times in A* algorithm for sliding puzzle problems.

In [ ]:
def h_manhattan(state: tuple, goal: tuple, goal_positions: dict = None, N: int = 3):
    if goal_positions is None:
        goal_positions = {tile: divmod(i, N) for i, tile in enumerate(goal)}
    
    distance = 0
    for i, tile in enumerate(state):
        if tile != 0:
            s_r, s_c = divmod(i, N)
            g_r, g_c = goal_positions[tile]
            distance += abs(s_r - g_r) + abs(s_c - g_c)

    return distance


In [11]:
start, goal = parse_astar_in(INPUT_FILE)

tracemalloc.start()
t0 = perf_counter()
path, N = astar_algorithm(start, goal, h_manhattan)

elapsed = perf_counter() - t0

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print_path(path, N=N)
print(f"time={elapsed:.4f}s  current={current/1e6:.2f}MB  peak={peak/1e6:.2f}MB")

P = 12 
Expanding: g=0  h=12  f=12
P = 13 
P = 13 
P = 11 
P = 11 
Expanding: g=1  h=11  f=12
P = 12 
P = 12 
Expanding: g=1  h=11  f=12
P = 10 
P = 10 
Expanding: g=2  h=10  f=12
P = 11 
Expanding: g=2  h=10  f=12
P = 9 
Expanding: g=3  h=9  f=12
P = 10 
P = 10 
Expanding: g=1  h=13  f=14
P = 12 
P = 12 
Expanding: g=1  h=13  f=14
P = 14 
P = 14 
Expanding: g=2  h=12  f=14
P = 11 
Expanding: g=2  h=12  f=14
P = 13 
Expanding: g=3  h=11  f=14
P = 12 
P = 10 
Expanding: g=4  h=10  f=14
P = 11 
P = 9 
P = 11 
Expanding: g=4  h=10  f=14
P = 11 
Expanding: g=2  h=12  f=14
P = 13 
Expanding: g=2  h=12  f=14
P = 13 
Expanding: g=3  h=11  f=14
P = 12 
P = 10 
Expanding: g=4  h=10  f=14
P = 11 
Expanding: g=5  h=9  f=14
P = 10 
P = 10 
Expanding: g=4  h=10  f=14
P = 11 
Expanding: g=2  h=14  f=16
P = 15 
Expanding: g=2  h=14  f=16
P = 15 
Expanding: g=3  h=13  f=16
P = 14 
P = 14 
Expanding: g=4  h=12  f=16
P = 13 
P = 11 
P = 11 
Expanding: g=5  h=11  f=16
P = 10 
P = 10 
Expanding: g=5  h=11

### Nilsson's Sequence Score

Nilsson's Sequence Score is a more sophisticated heuristic that combines sequence scoring with Manhattan distance. It evaluates the puzzle state by considering both the positions of tiles and their ordering in rows and columns. This heuristic assigns penalties for tiles that are out of sequence and adds the Manhattan distance component. While more computationally expensive than simpler heuristics, it provides a tighter bound and can significantly reduce the search space in A* algorithm for certain puzzle configurations.

In [ ]:
def h_nilsson(state: tuple, goal: tuple, goal_positions: dict = None, N: int = 3):
    manhattan_distance = h_manhattan(state, goal, goal_positions, N)

    # Ring order initialization: 0,1,2,5,8,7,6,3 (clockwise from top-left corner)
    ring_order = [0, 1, 2, 5, 8, 7, 6, 3]

    sequence_score = 0

    # Check the ring order for misplaced tiles (ignoring the blank tile)
    for i in range(len(ring_order)):
        curr_tile = state[ring_order[i]]

        # Ignore the blank tile when calculating sequence score
        if curr_tile == 0:
            continue

        # expected successor of this tile in goal state
        # In 8-puzzle, successor is tile+1 except 8 -> 1
        expected_successor = 1 if curr_tile == 8 else curr_tile + 1

        next_tile = state[ring_order[(i + 1) % len(ring_order)]]

        if next_tile != expected_successor:
            sequence_score += 2

    # center penalty
    if state[4] != 0:
        sequence_score += 1
    
    # The sequence score is multiplied by 3 to give it more weight compared to the Manhattan distance.
    # f(n) = g(n) + h(n) where h(n) = P(n) + 3 * S(n)
    return manhattan_distance + 3 * sequence_score

In [13]:
start, goal = parse_astar_in(INPUT_FILE)

tracemalloc.start()
t0 = perf_counter()
path, N = astar_algorithm(start, goal, h_nilsson)

elapsed = perf_counter() - t0

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print_path(path, N=N)
print(f"time={elapsed:.4f}s  current={current/1e6:.2f}MB  peak={peak/1e6:.2f}MB")

P = 12 
S = 16
Expanding: g=0  h=60  f=60
P = 13 
S = 15
P = 13 
S = 15
P = 11 
S = 15
P = 11 
S = 15
Expanding: g=1  h=56  f=57
P = 12 
S = 15
P = 12 
S = 15
Expanding: g=1  h=56  f=57
P = 10 
S = 15
P = 10 
S = 15
Expanding: g=2  h=55  f=57
P = 11 
S = 15
Expanding: g=2  h=55  f=57
P = 9 
S = 15
Expanding: g=3  h=54  f=57
P = 10 
S = 16
P = 10 
S = 15
Expanding: g=1  h=58  f=59
P = 12 
S = 15
P = 12 
S = 15
Expanding: g=1  h=58  f=59
P = 14 
S = 15
P = 14 
S = 15
Expanding: g=2  h=57  f=59
P = 11 
S = 15
Expanding: g=2  h=57  f=59
P = 13 
S = 15
Expanding: g=3  h=56  f=59
P = 12 
S = 14
P = 10 
S = 15
Expanding: g=4  h=54  f=58
P = 13 
S = 13
P = 11 
S = 13
P = 11 
S = 13
Expanding: g=5  h=50  f=55
P = 12 
S = 13
P = 12 
S = 13
Expanding: g=5  h=50  f=55
P = 12 
S = 15
P = 10 
S = 13
Expanding: g=6  h=49  f=55
P = 9 
S = 13
Expanding: g=7  h=48  f=55
P = 8 
S = 10
P = 10 
S = 13
Expanding: g=8  h=38  f=46
P = 7 
S = 11
P = 7 
S = 9
P = 9 
S = 9
Expanding: g=9  h=34  f=43
P = 8 
S = 9